# Обучение агента в Unity ML-Agents с экспортом ONNX

Этот ноутбук обучает агента из Unity-среды (`MyAgent?team=0`) с 6 наблюдениями, 2 непрерывными действиями и 1 дискретным действием. Используем PPO из `stable-baselines3` и экспортируем модель в ONNX для Unity.

## Зависимости
- Установите библиотеки:
  ```bash
  pip install mlagents==0.30.0 stable-baselines3==2.0.0 gymnasium==0.28.1 torch==2.0.1 onnx numpy psutil
  ```
- Убедитесь, что путь к `UnityEnvironment.exe` правильный.
- Среда Unity должна быть собрана с `Behavior Name: MyAgent?team=0`.
- Python 3.8, совместимый с `mlagents==0.30.0`.

In [5]:
pip install mlagents==0.30.0 stable-baselines3==2.0.0 gymnasium==0.28.1 torch==2.0.1 onnx numpy psutil

Note: you may need to restart the kernel to use updated packages.


In [1]:
# Импорт библиотек
import os
import numpy as np
import torch
import torch.nn as nn
from mlagents_envs.environment import UnityEnvironment
from mlagents_envs.side_channel.engine_configuration_channel import EngineConfigurationChannel
from mlagents_envs.base_env import ActionTuple
from stable_baselines3 import PPO
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
from stable_baselines3.common.env_checker import check_env
import gymnasium as gym
from gymnasium import spaces

# Функция закрытия среды
def close_unity_env(env):
    try:
        if env is not None:
            env.close()
            print('Среда Unity успешно закрыта.')
        else:
            print('Среда не инициализирована.')
    except Exception as e:
        print(f'Ошибка при закрытии среды: {e}')
    finally:
        import psutil
        for proc in psutil.process_iter(['name']):
            if proc.info['name'].lower() == 'unityenvironment.exe':
                proc.kill()
                print(f'Процесс UnityEnvironment.exe (PID: {proc.pid}) принудительно завершён.')
            if 'unity' in proc.info['name'].lower():
                proc.kill()
                print(f"Процесс {proc.info['name']} (PID: {proc.pid}) принудительно завершён.")

# Путь к среде Unity
env_path = os.path.join(os.getcwd(), r'N:\MyRL\My_First_NPC\MyfirstMPC\UnityEnvironment.exe')

# Настройка канала для ускорения симуляции
engine_channel = EngineConfigurationChannel()
engine_channel.set_configuration_parameters(time_scale=50.0, quality_level=0)

# Инициализация среды
try:
    env = UnityEnvironment(file_name=env_path, worker_id=1, base_port=6000, side_channels=[engine_channel], timeout_wait=60)
    env.reset()
except Exception as e:
    print(f'Ошибка инициализации среды: {e}')
    close_unity_env(None)
    raise

# Получение имени поведения
behavior_name = list(env.behavior_specs.keys())[0]
print(f'Behavior Name: {behavior_name}')
spec = env.behavior_specs[behavior_name]

# Проверка спецификации
print(f'Observation size: {spec.observation_specs[0].shape[0]}')
print(f'Continuous action size: {spec.action_spec.continuous_size}')
print(f'Discrete action branches: {spec.action_spec.discrete_branches}')

Behavior Name: MyAgent?team=0
Observation size: 6
Continuous action size: 2
Discrete action branches: (1,)


## Создание кастомной нейросети

Нейросеть для 6 наблюдений, 2 непрерывных и 1 дискретного действия.

In [2]:
class CustomActorCriticNet(BaseFeaturesExtractor):
    def __init__(self, observation_space, features_dim=128):
        super(CustomActorCriticNet, self).__init__(observation_space, features_dim)
        self.fc1 = nn.Linear(observation_space.shape[0], 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, features_dim)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

policy_kwargs = dict(
    features_extractor_class=CustomActorCriticNet,
    features_extractor_kwargs=dict(features_dim=128),
    net_arch=[dict(pi=[64, 32], vf=[64, 32])]
)

## Обёртка среды для Gymnasium

Используем `Box` для непрерывных действий и обрабатываем дискретные внутри `step`.

In [3]:
class UnityGymWrapper(gym.Env):
    def __init__(self, unity_env, behavior_name, spec):
        super(UnityGymWrapper, self).__init__()
        self.env = unity_env
        self.behavior_name = behavior_name
        self.spec = spec

        # Observation space
        self.observation_space = spaces.Box(
            low=-np.inf, high=np.inf, shape=(spec.observation_specs[0].shape[0],), dtype=np.float32
        )

        # Action space: only continuous actions (2) in Box
        self.action_space = spaces.Box(
            low=-1.0, high=1.0, shape=(spec.action_spec.continuous_size,), dtype=np.float32
        )
        # Discrete action will be handled manually
        self.discrete_branches = spec.action_spec.discrete_branches

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.env.reset()
        decision_steps, _ = self.env.get_steps(self.behavior_name)
        obs = decision_steps.obs[0][0]
        info = {}
        return obs, info

    def step(self, action):
        # Ensure action is 2D: (num_agents, action_size)
        continuous_action = np.asarray(action, dtype=np.float32).reshape(1, -1)
        # Discrete action: threshold based on first continuous action
        discrete_action = np.array([[1 if action[0] > 0 else 0]], dtype=np.int32)  # shape (1, num_discrete_branches)

        action_tuple = ActionTuple()
        action_tuple.add_continuous(continuous_action)
        action_tuple.add_discrete(discrete_action)

        self.env.set_actions(self.behavior_name, action_tuple)
        self.env.step()

        decision_steps, terminal_steps = self.env.get_steps(self.behavior_name)
        if len(terminal_steps) > 0:
            done = True
            reward = float(terminal_steps.reward[0])
            obs = terminal_steps.obs[0][0]
        else:
            done = False
            reward = float(decision_steps.reward[0])
            obs = decision_steps.obs[0][0]
        info = {}
        truncated = False

        return obs, reward, done, truncated, info

    def close(self):
        close_unity_env(self.env)

gym_env = UnityGymWrapper(env, behavior_name, spec)
check_env(gym_env)

## Обучение модели

Используем PPO для обучения.

In [4]:
try:
    model = PPO(
        'MlpPolicy',
        gym_env,
        policy_kwargs=policy_kwargs,
        learning_rate=3e-4,
        n_steps=2048,
        batch_size=64,
        n_epochs=10,
        gamma=0.99,
        gae_lambda=0.95,
        clip_range=0.2,
        ent_coef=0.01,
        verbose=1
    )
    model.learn(total_timesteps=10)
    model.save('ppo_myagent')
except Exception as e:
    print(f'Ошибка обучения: {e}')
finally:
    close_unity_env(env)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


c:\Users\User\.conda\envs\ML_Agents\lib\site-packages\stable_baselines3\common\policies.py:460: UserWarning: As shared layers in the mlp_extractor are removed since SB3 v1.8.0, you should now pass directly a dictionary and not a list (net_arch=dict(pi=..., vf=...) instead of net_arch=[dict(pi=..., vf=...)])
  warnings.warn(


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 97.2     |
|    ep_rew_mean     | -0.552   |
| time/              |          |
|    fps             | 302      |
|    iterations      | 1        |
|    time_elapsed    | 6        |
|    total_timesteps | 2048     |
---------------------------------
Среда Unity успешно закрыта.
Процесс Unity Hub.exe (PID: 107332) принудительно завершён.
Процесс Unity.exe (PID: 196792) принудительно завершён.
Процесс Unity Hub.exe (PID: 222720) принудительно завершён.
Процесс Unity Hub.exe (PID: 223204) принудительно завершён.
Процесс UnityCrashHandler64.exe (PID: 223496) принудительно завершён.
Процесс UnityAutoQuitter.exe (PID: 228864) принудительно завершён.
Процесс Unity.exe (PID: 230528) принудительно завершён.
Процесс Unity.Licensing.Client.exe (PID: 230580) принудительно завершён.
Процесс Unity.exe (PID: 231752) принудительно завершён.
Процесс Unity.ILPP.Runner.exe (PID: 231912) принудительно завершён.
Проце

## Экспорт модели в ONNX

Экспортируем модель в ONNX для Unity Barracuda.

In [5]:
import torch
from torch.nn import Parameter

# Кастомная обёртка для экспорта в ONNX с поддержкой Sentis
class WrapperNet(torch.nn.Module):
    def __init__(self, policy, continuous_action_size):
        """
        Оборачивает политику PPO, добавляя тензоры для Unity ML-Agents Sentis.
        :param policy: Политика PPO из stable-baselines3.
        :param continuous_action_size: Размер непрерывных действий (2 для MyAgent).
        """
        super(WrapperNet, self).__init__()
        self.policy = policy

        # version_number: MLAgents2_0 = 3
        version_number = torch.tensor([3], dtype=torch.float32)
        self.version_number = Parameter(version_number, requires_grad=False)

        # memory_size: 0, так как нет RNN
        memory_size = torch.tensor([0], dtype=torch.float32)
        self.memory_size = Parameter(memory_size, requires_grad=False)

        # continuous_action_output_shape: [2] для 2 непрерывных действий
        continuous_shape = torch.tensor([continuous_action_size], dtype=torch.float32)
        self.continuous_shape = Parameter(continuous_shape, requires_grad=False)

    def forward(self, obs, mask):
        """
        Прямой проход: вычисляет непрерывные действия и возвращает дополнительные тензоры.
        :param obs: Наблюдения [batch_size, 6].
        :param mask: Фиктивный вход для action_masks [batch_size, 2].
        :return: continuous_actions, continuous_shape, version_number, memory_size.
        """
        # Получаем непрерывные действия из политики
        continuous_actions = self.policy(obs, deterministic=True)[0]
        # Умножаем на mask (фиктивно, так как маски не используются)
        continuous_actions = torch.mul(continuous_actions, mask)
        return continuous_actions, self.continuous_shape, self.version_number, self.memory_size

try:
    # Переводим политику на CPU
    policy = model.policy.to('cpu')
    continuous_action_size = spec.action_spec.continuous_size  # 2 для MyAgent
    wrapper_net = WrapperNet(policy, continuous_action_size)
    
    # Пример входных тензоров
    dummy_input = torch.randn(1, spec.observation_specs[0].shape[0])  # [1, 6]
    dummy_mask = torch.ones(1, continuous_action_size)  # [1, 2], фиктивная маска
    
    # Экспорт в ONNX
    torch.onnx.export(
        wrapper_net,
        (dummy_input, dummy_mask),
        'trained_myagent.onnx',
        input_names=['obs_0', 'action_masks'],
        output_names=['continuous_actions', 'continuous_action_output_shape', 'version_number', 'memory_size'],
        dynamic_axes={
            'obs_0': {0: 'batch'},
            'action_masks': {0: 'batch'},
            'continuous_actions': {0: 'batch'},
            'continuous_action_output_shape': {0: 'batch'},
            'version_number': {0: 'batch'},
            'memory_size': {0: 'batch'}
        },
        opset_version=9,  # Пробуем opset 9 для Unity ML-Agents 2.0.1
        verbose=False
    )
    print('Модель успешно сохранена: trained_myagent.onnx')
    print('Файл существует:', os.path.exists('trained_myagent.onnx'))
except Exception as e:
    print(f'Ошибка экспорта ONNX: {e}')
    print('Попробуйте opset_version=11 или проверьте версию Unity Sentis.')
finally:
    close_unity_env(env)

============== Diagnostic Run torch.onnx.export version 2.0.1+cpu ==============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

Модель успешно сохранена: trained_myagent.onnx
Файл существует: True
Ошибка при закрытии среды: No Unity environment is loaded.


## Тестирование модели

Проверяем, как работает обученная модель.

In [ ]:
try:
    obs, _ = gym_env.reset()
    for _ in range(1000):
        action, _ = model.predict(obs, deterministic=True)
        obs, reward, done, truncated, info = gym_env.step(action)
        if done or truncated:
            print('Эпизод завершён')
            obs, _ = gym_env.reset()
except Exception as e:
    print(f'Ошибка тестирования: {e}')
finally:
    close_unity_env(env)